## 5.1.1 Phép toán Convolution 

In [2]:
HTML('<img src="https://d3ansictanv2wj.cloudfront.net/Figure_10-b758680d6af48163e334214af0513546.gif"/>')

Điểm khác biệt giữa Dense layer và Conv layer là :
- Dense layer sẽ học đặc điểm của toàn thể ảnh
- Conv layer học đặc điểm của từng cụm trên ảnh  

Đặc điểm chính đó giúp convnets có những tính chất thú vị sau :
- Những tính chất mà convnet học được là **bất biến** (translation invariant) : sau khi học 1 số thuộc tính ở góc trái của ảnh, convnet có thể **nhận ra thuộc tính đó ở bất cứ mọi nơi trên ảnh**. Trong khi đó **Dense layer sẽ phải học lại** khi thuộc tính xuất hiện ở **vị trí khác**. --> Cần ít training examples hơn nhưng độ bao quát của model CNN lại lớn hơn
- Các **thuộc tính** của object được **convnet** học và **phát triển qua mỗi layer** : Layer đầu sẽ học thuộc tính các **chi tiết nhỏ**, như cạnh, đường cong,... layer sau sẽ học các thuộc tính lớn hơn nhờ tổng hợp từ layer trước --> Convnets có thể học các thuộc tính của object **phức tạp**

Hình dung quá trình học của convnet qua các layer :

![learn-cat](cat.png)

                                        FChollet's Deep learning book

Phép toán convolution trên các 3D tensors, được gọi là "feature maps", **3D** ở đây gồm 2 trục (**height, width**) và 1 chiều sâu (**depth**, hay còn gọi channels - số kênh màu). Với ảnh RGB, depth = 3 tượng trưng 3 kênh màu R,G,B. Còn ảnh đen trắnng ( như MNIST ) số kênh màu là 1.

Phép toán convolution sẽ tách các cụm nhỏ từ input feature map, sau đó thực hiện 1 phép **transform** với tất cả các cụm đó theo từng kernel, tạo nên 1 output feature map.
Output đó vẫn có **shape là 3D**, vẫn có width và height, còn depth là 1 param của layer, không còn đại diện cho số kênh màu nữa, thay vào đó là số "**filters**". Filter, hay còn gọi là kernel, là **vector các trọng số tượng** trưng cho 1 thuộc tính của ảnh 
> "In the context of CNN, a filter is a set of **learnable weights** which are **learned using the backpropagation** algorithm. You can think of each filter as **storing a single template/pattern**. When you convolve this filter across the corresponding input, you are basically trying to **find out the similarity** between the stored template and different locations in the input." [Quora](https://www.quora.com/What-is-a-filter-in-the-context-of-Convolutional-Neural-Networks/answers/25755168)

#### Input feature (Image) sau khi thực hiện phép toán Convolution với kernel [[1,0,1],[0,1,0],[1,0,1]] 

Trở lại với bài toán MNIST ta đã làm bằng convnet, model của chúng ta như sau :
```
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320        (28,28,1) --> (26,26,32) : 32filters
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
=================================================================
Total params: 93,322
Trainable params: 93,322
Non-trainable params: 0
```

```python
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# size của filter là 3x3, có 32 filter
```

Ở layer đầu, input feature map có shape là (28,28,1) (ảnh 28 * 28, với 1 kênh màu), và output feature map có shape là (26,26,32). Điều này có nghĩa là sẽ có 32 filter được tạo ra, mỗi filter là ma trận 3x3

Convolution Có 2 parameter chính :
- size của cụm pixel được tách ra từ inputs ( thường là 3x3 hoặc 5x5 ), cũng là size của kernel. Trong ví dụ trước ta đã dùng 3x3 - một size khá phổ biến 
- depth của ouput feature map ( số filters cho phép convolutions ). Trong ví dụ ta dùng depth lần lượt là 32 và 64 

Trong Keras, ta dùng câu lệnh sau để gọi 1 convolutional 2D layer
```python
Conv2D(output_depth,(window_height,window_width))
```

In [6]:
from IPython.display import HTML
HTML('<img src="https://cdn-images-1.medium.com/max/1600/1*ZCjPUFrB6eHPRi4eyP6aaA.gif"/>')

Ảnh trên : Mỗi ô giá trị của convolved feature là **tổng của tích tương ứng** mỗi pixel của từng **sliding windows** 3x3 với kernel
```
[[1,0,1],
[0,1,0],
[1,0,1]]
```
Lưu ý, đối với ảnh RGB hoặc với input map có **depth >1** : Mỗi ô giá trị của convolved feature là **tổng các giá trị** thu được **ở mỗi channel** của input

**Cách hoạt động** của phép convolution : Ta sẽ lấy từng cụm pixel có size là 3x3 (hoặc 5x5) từ input map (hiểu đơn giản như sliding windows), ta sẽ có được 1 **khối 3D** (window_width, window_height, input_depth). Mỗi khối 3D này sẽ thực hiện transform với tất cả kernel, trả về **1D vector** với **shape (output_depth)**. Tất cả những vector đó sẽ **hợp lại theo thứ tự** tạo thành 1 khối **3D output_map** với shape (height, width, output_depth), quá trình đó được tổng kết trong hình sau :

![Convolution](cnn.png)

                                    FChollet's Deep learning with python

Lưu ý rằng output_width và height sẽ khác so với input, phụ thuộc vào 2 lý do sau :
- Border effect 
- Strides

#### BORDER EFFECT
Ảnh của bạn có size 5x5, ta sử dụng kernel có size 3x3, các windows cách nhau 1 pixel --> Có tất cả (5-3+1)*(5-3+1) = 9 cụm ảnh 3x3 có thể tách ra.
Sau phép convolution với kernel và ghép các giá trị thu được lại, ta có 1 ma trận ảnh mới có size 3x3 (Ảnh động bên trên). Như vậy, size của ảnh đã thay đổi 5x5 --> 3x3

Trong ví dụ MNIST, ảnh ban đầu có size 28x28, sau layer convolution đầu tiên có kernel 3x3, ta thu được output có size là (28-3+1)x(28-3+1) = 26x26  

Padding : Thêm 1 số lượng hàng và cột nhất định vào xung quanh input

Trong Keras : padding có 2 arguments :
- "valid" : không padding --> default
- "same" : padding để input và output có cùng width & height

#### CONVOLUTION STRIDES
Stride = số ô giữa 2 tâm của 2 windows liền nhau + **1**

e.g : Với input có size = 7, kernel size = 3 :
- stride = 1 --> (7-3)/1+1 = 5 --> size out_put 5x5
- stride = 2 --> (7-3)/2+1 = 3 --> size out_put 3x3
Công thức tính size của output feature :

```python
def output_size(input_size, kernel_size, stride) :
    assert (input_size - kernel_size) % stride == 0
    return int((input_size -kernel_size)/stride)+1
```
Lưu ý : size - stride luôn phải là bội số của kernel   

Stride > 1 được sử dụng để giảm số mẫu cho network, tuy vậy cũng làm giảm độ chính xác của model. Thay vào đó ta sử dụng max-pooling mà chúng ta đã thấy trong ví dụ MNIST.

Link tham khảo :
- Kernel - Hiểu hơn về kernel [Hit Or Miss Opencv](https://docs.opencv.org/master/db/d06/tutorial_hitOrMiss.html)
- Convolution neural network - Udacity [youtube](https://youtu.be/ISHGyvsT0QY) - Vi deo rất trực quan
- Convolution neural network Intel neon (khá hay) [youtube](https://www.youtube.com/watch?v=xg2ajb3csgk)
- Cs231n - Rất chi tiết [link](http://cs231n.github.io/convolutional-networks/#conv)